<a href="https://colab.research.google.com/github/asjad895/GENAI_CHATBOT/blob/main/Research/llama_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torch
!pip install transformers
!pip install accelerate

In [2]:
class PromptTemplate:
    def __init__(self,chatbot_prompt :str = None, intent_classifier_prompt :str = None) ->None:
      self.chatbot_prompt = """
""" if chatbot_prompt is None else chatbot_prompt

      self.intent_classifier_prompt = """
""" if intent_classifier_prompt is None else intent_classifier_prompt

    def format(self,chatbot_prompt = True,**kwargs) ->str:
        if chatbot_prompt:
            # knowledge_source = kwargs['knowledge_source']
            # del kwargs['knowledge_source']
             system = {"role":"system","content":f"{self.chatbot_prompt}"}
        else:
            system = {"role":"system","content":f"{self.intent_classifier_prompt}"}

        return system


In [3]:
domain_examples = {
    "Healthcare": [
        {"query": "I have a headache and a fever, what could it be?", "intent": "Symptom Check"},
        {"query": "My throat is really sore and I'm coughing, what should I do?", "intent": "Symptom Check"},
        {"query": "I woke up with a rash all over my body, is this contagious?", "intent": "Symptom Check"},
        {"query": "I've been feeling dizzy and nauseous lately, what could be the cause?", "intent": "Symptom Check"},
        {"query": "Can I book an appointment with Dr. Smith next Tuesday?", "intent": "Doctor Appointment"},
        {"query": "I need to see a cardiologist, how do I schedule a consultation?", "intent": "Doctor Appointment"},
        {"query": "What are the available slots for a pediatric dentist next week?", "intent": "Doctor Appointment"},
        {"query": "I need to reschedule my appointment with Dr. Jones, what should I do?", "intent": "Doctor Appointment"},
        {"query": "What are the side effects of ibuprofen?", "intent": "Medication Information"},
        {"query": "Can I take this medication with antibiotics?", "intent": "Medication Information"},
        {"query": "How long does it take for this medication to start working?", "intent": "Medication Information"},
        {"query": "What is the dosage for this medicine for children?", "intent": "Medication Information"},
        {"query": "What are the visiting hours for XYZ Hospital?", "intent": "Hospital Information"},
        {"query": "Is there a pharmacy in ABC Hospital?", "intent": "Hospital Information"},
        {"query": "How do I get to DEF Hospital from my location?", "intent": "Hospital Information"},
        {"query": "Does this hospital have an emergency department?", "intent": "Hospital Information"},
        {"query": "What are the different types of health checkups available?", "intent": "Health Checkup"},
        {"query": "How much does a full body checkup cost?", "intent": "Health Checkup"},
        {"query": "Does your health insurance cover preventive checkups?", "intent": "Health Checkup"},
        {"query": "I want to book a diabetes screening test, how do I proceed?", "intent": "Health Checkup"},
    ],
    "Insurance": [
        {"query": "What is the coverage for my car insurance?", "intent": "Policy Information"},
        {"query": "Does my health insurance cover pre-existing conditions?", "intent": "Policy Information"},
        {"query": "What are the different types of life insurance policies available?", "intent": "Policy Information"},
        {"query": "What is the deductible for my home insurance?", "intent": "Policy Information"},
        {"query": "How do I file a claim for a car accident?", "intent": "Claim Filing"},
        {"query": "What documents are required to file a health insurance claim?", "intent": "Claim Filing"},
        {"query": "My car was stolen, how do I file a claim?", "intent": "Claim Filing"},
        {"query": "I need to submit a claim for a dental procedure, what are the steps?", "intent": "Claim Filing"},
        {"query": "What are the different payment options for my insurance premium?", "intent": "Premium Payment"},
        {"query": "When is my next premium due?", "intent": "Premium Payment"},
        {"query": "Can I pay my premium online?", "intent": "Premium Payment"},
        {"query": "What happens if I miss a premium payment?", "intent": "Premium Payment"},
        {"query": "Is physiotherapy covered under my health insurance?", "intent": "Coverage Check"},
        {"query": "Does my travel insurance cover lost luggage?", "intent": "Coverage Check"},
        {"query": "Is cosmetic surgery covered by my health plan?", "intent": "Coverage Check"},
        {"query": "Will my car insurance cover damage caused by a flood?", "intent": "Coverage Check"},
    ],
    "Finance": [
        {"query": "What are the best mutual funds to invest in for long-term growth?", "intent": "Investment Advice"},
        {"query": "Should I invest in stocks or bonds right now?", "intent": "Investment Advice"},
        {"query": "What are the risks and rewards of investing in cryptocurrency?", "intent": "Investment Advice"},
        {"query": "How can I start investing in the stock market?", "intent": "Investment Advice"},
        {"query": "What are the eligibility criteria for a personal loan?", "intent": "Loan Application"},
        {"query": "How do I apply for a home loan online?", "intent": "Loan Application"},
        {"query": "What documents are required for a car loan application?", "intent": "Loan Application"},
        {"query": "What is the interest rate on a student loan?", "intent": "Loan Application"},
        {"query": "My credit card is not working, what should I do?", "intent": "Credit Card Issues"},
        {"query": "How do I dispute a transaction on my credit card?", "intent": "Credit Card Issues"},
        {"query": "What are the fees associated with my credit card?", "intent": "Credit Card Issues"},
        {"query": "I suspect fraudulent activity on my credit card, what steps should I take?", "intent": "Credit Card Issues"},
        {"query": "How can I save money on my groceries?", "intent": "Savings & Budgeting"},
        {"query": "What are some tips for creating a budget?", "intent": "Savings & Budgeting"},
        {"query": "How can I reduce my monthly expenses?", "intent": "Savings & Budgeting"},
        {"query": "What are some high-yield savings accounts?", "intent": "Savings & Budgeting"},
    ],
    "Retail": [
        {"query": "Is the new Samsung phone available in stock?", "intent": "Product Availability"},
        {"query": "Do you have any red t-shirts in size medium?", "intent": "Product Availability"},
        {"query": "Is this product available for delivery to my address?", "intent": "Product Availability"},
        {"query": "When will this item be back in stock?", "intent": "Product Availability"},
        {"query": "What is the price of this laptop?", "intent": "Product Pricing"},
        {"query": "Are there any discounts available on this item?", "intent": "Product Pricing"},
        {"query": "What is the price difference between these two models?", "intent": "Product Pricing"},
        {"query": "Do you offer any price matching?", "intent": "Product Pricing"},
        {"query": "How do I track my order?", "intent": "Order Tracking"},
        {"query": "What is the status of my delivery?", "intent": "Order Tracking"},
        {"query": "When can I expect my order to arrive?", "intent": "Order Tracking"},
        {"query": "Why is my order delayed?", "intent": "Order Tracking"},
        {"query": "How do I return an item?", "intent": "Returns & Refunds"},
        {"query": "What is the return policy for this product?", "intent": "Returns & Refunds"},
        {"query": "How long does it take to get a refund?", "intent": "Returns & Refunds"},
        {"query": "Can I exchange this item for a different size?", "intent": "Returns & Refunds"},
    ]
}

In [4]:
import pandas as pd
examples_df = pd.DataFrame(columns=["Intent", "Examples"])
for domain, examples in domain_examples.items():
    for example in examples:
        examples_df = pd.concat([examples_df, pd.DataFrame({"Intent": [domain.lower().strip()], "Examples": [example["query"]]})], ignore_index=True)

print(examples_df.shape)
examples_df.head()


(68, 2)


,Intent,Examples
0,healthcare,"I have a headache and a fever, what could it be?"
1,healthcare,"My throat is really sore and I'm coughing, wha..."
2,healthcare,"I woke up with a rash all over my body, is thi..."
3,healthcare,"I've been feeling dizzy and nauseous lately, w..."
4,healthcare,Can I book an appointment with Dr. Smith next ...


In [5]:
from jinja2 import Template

In [6]:
intents_des = """Healthcare: This domain focuses on health and medical-related queries. The chatbot must understand and address concerns
related to symptoms, diagnoses, treatments, medications, doctor or specialist information, health check-up packages,
and appointment scheduling. It should also handle questions about healthcare services like telemedicine, hospital locations, or insurance-covered treatments.

Insurance : This domain addresses questions about insurance policies, claims, premiums, coverage, and renewals. It should provide information on types of insurance (health, life, vehicle, etc.),
coverage inclusions and exclusions, and process-oriented queries like filing claims or resolving disputes.

Finance: This domain caters to financial-related queries such as investment options, bank loans, credit card issues, or savings plans. The chatbot must address personal finance management, tax-saving methods, account setup, and transactional problems while maintaining
contextual understanding.

Retail: This domain involves queries about shopping, products, pricing, availability, discounts, order status, returns,refund and customer support. It also includes guidance on using e-commerce platforms or stores to make purchases and track orders.

other: This should trigger when user query is out of above domain means user query is not falling to any of above domains(Healthcare,Insurance,Finance,Retail).
"""

intents_prompt = """--Role--
You are an expert in intent classifier of user query to the only given domain.
These are the domain with description you should classify the query into one of these domain only-
{{intents}}
--Restrictions--
Do not generate anything apart from json output.
--output--
Always return output in this format-
{"intent":"your prediction"}
"""

In [7]:
expected_intents = ['healthcare','insurance','finance','retail','other']

In [8]:
template = Template(intents_prompt)
prompt = template.render(intents=intents_des)
print(template.render(intents=intents_des))

--Role--
You are an expert in intent classifier of user query to the only given domain.
These are the domain with description you should classify the query into one of these domain only-
Healthcare: This domain focuses on health and medical-related queries. The chatbot must understand and address concerns
related to symptoms, diagnoses, treatments, medications, doctor or specialist information, health check-up packages,
and appointment scheduling. It should also handle questions about healthcare services like telemedicine, hospital locations, or insurance-covered treatments.

Insurance : This domain addresses questions about insurance policies, claims, premiums, coverage, and renewals. It should provide information on types of insurance (health, life, vehicle, etc.),
coverage inclusions and exclusions, and process-oriented queries like filing claims or resolving disputes.

Finance: This domain caters to financial-related queries such as investment options, bank loans, credit card issue

In [9]:
prompt_template = PromptTemplate(intent_classifier_prompt=prompt)
system_message = prompt_template.format(chatbot_prompt=False,intent_classifier_prompt=True)
print(prompt_template.format(chatbot_prompt=False,intent_classifier_prompt=True))

{'role': 'system', 'content': '--Role--\nYou are an expert in intent classifier of user query to the only given domain.\nThese are the domain with description you should classify the query into one of these domain only-\nHealthcare: This domain focuses on health and medical-related queries. The chatbot must understand and address concerns\nrelated to symptoms, diagnoses, treatments, medications, doctor or specialist information, health check-up packages,\nand appointment scheduling. It should also handle questions about healthcare services like telemedicine, hospital locations, or insurance-covered treatments.\n\nInsurance : This domain addresses questions about insurance policies, claims, premiums, coverage, and renewals. It should provide information on types of insurance (health, life, vehicle, etc.),\ncoverage inclusions and exclusions, and process-oriented queries like filing claims or resolving disputes.\n\nFinance: This domain caters to financial-related queries such as investme

In [10]:
chatbot = """
"""

In [11]:
from typing import List,Dict
class ChatHistory:
  def __init__(self) -> None:
      self.chat_history = []


  def add_message(self,role : str,content : str) -> None:
      self.chat_history.append({"role":role,"content":content})

  def get_chat_history(self) -> List[Dict]:
      return self.chat_history

  def clear_chat_history(self) -> None:
      self.chat_history = []

  def __len__(self) -> int:
      return len(self.chat_history)



In [19]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
%%capture
!pip install bitsandbytes

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
from typing import Tuple,Union,List,Dict
import subprocess

class ChatCompletion:
  def __init__(self ,**kwargs) -> None:
    if 'device' in kwargs:
      self.device = kwargs['device']
    else:
      self.device = "cuda:0" if torch.cuda.is_available() else "cpu"

    if 'model' in kwargs:
      try:
        self.model = kwargs['model']
        self.tokenizer = kwargs['tokenizer']
      except:
        raise Exception("'model' or 'tokenizer' not found")

  def create(self,messages : List[Dict],**kwargs) -> str:
    if 'max_new_tokens' in kwargs:
      max_new_tokens = kwargs['max_new_tokens']
    else:
      max_new_tokens = 500
    if 'temperature' in kwargs:
      temperature = kwargs['temperature']
    else:
      temperature = 0.6
    if 'top_p' in kwargs:
      top_p = kwargs['top_p']
    else:
      top_p = 0.9

    if 'do_sample' in kwargs:
      do_sample = kwargs['do_sample']
    else:
      do_sample = False

    if 'repetition_penalty' in kwargs:
      repetition_penalty = kwargs['repetition_penalty']
    else:
      repetition_penalty = 1.0

    if 'eos_token_id' in kwargs:
      eos_token_id = kwargs['eos_token_id']

    else:
      eos_token_id = None

    if 'pad_token_id' in kwargs:
      pad_token_id = kwargs['pad_token_id']
    else:
      pad_token_id = self.tokenizer.eos_token_id

    if 'return_full_text' in kwargs:
      return_full_text = kwargs['return_full_text']
    else:
      return_full_text = False

    if 'return_prompt' in kwargs:
      return_prompt = kwargs['return_prompt']
    else:
      return_prompt = False

    if 'return_instruction' in kwargs:
      return_instruction = kwargs['return_instruction']
    else:
      return_instruction = False

    encoded_input = tokenizer.apply_chat_template(messages, tokenize = True,return_dict=True,return_tensors="pt").to(self.device)
    outputs = model.generate(
        **encoded_input,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
       )
    response = tokenizer.decode(outputs[0])
    response = response.split('assistant<|end_header_id|>')[-1].strip().split('<|eot_id|>')[0].strip()
    return response



In [46]:
# unsloth/Llama-3.3-70B-Instruct-GGUF
# meta-llama/Llama-3.2-11B-Vision-Instruct
# mistralai/Mistral-7B-Instruct-v0.3

In [47]:
# unsloth/Llama-3.3-70B-Instruct-GGUF

In [48]:
import torch
import subprocess
from functools import wraps
from typing import Tuple
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import os
save_dir = 'loaded_model'

def gpu_memory_decorator(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        before_memory = self._get_gpu_memory()
        print(f"Available GPU Memory Before '{func.__name__}': {before_memory:.2f} MB")

        result = func(self, *args, **kwargs)

        after_memory = self._get_gpu_memory()
        print(f"Available GPU Memory After '{func.__name__}': {after_memory:.2f} MB")

        return result
    return wrapper

class LLM:
    def __init__(self, **kwargs) -> None:
        if 'model_id' in kwargs:
            self.model_id = kwargs['model_id']
        else:
            self.model_id = "meta-llama/Llama-3.2-3b-Instruct"

        if 'device' in kwargs:
            self.device = kwargs['device']
        else:
            self.device = "cuda:0" if torch.cuda.is_available() else "cpu"

    def __call__(self) -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
        model, tokenizer = self.__get_model(self.model_id)
        return model, tokenizer

    @gpu_memory_decorator
    def __get_model(self, model_id: str = 'meta-llama/Llama-3.2-3B-Instruct') -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
        if self._get_gpu_memory() <= 60000:
          print("Loading Quantized Model")
          if os.path.exists(save_dir) and os.path.isdir(save_dir):
            print(f"Model found in '{save_dir}' directory. Loading model from there...")
            model = AutoModelForCausalLM.from_pretrained(save_dir)
            tokenizer = AutoTokenizer.from_pretrained(save_dir)
          else:
            quantization_config = BitsAndBytesConfig(load_in_8bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.bfloat16)
            print(f"Model not found in '{save_dir}' directory. Loading from the model ID: {model_id}...")
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                device_map="auto",
                torch_dtype=torch.bfloat16,
                quantization_config=quantization_config
              )
            # Load the tokenizer
            tokenizer = AutoTokenizer.from_pretrained(model_id)

          model.save_pretrained(save_dir, from_pt=True)
          tokenizer.save_pretrained(save_dir)

          return model, tokenizer
        else:
          print("Loading UnQuantized Model")
          model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16)
          tokenizer = AutoTokenizer.from_pretrained(model_id)
          return model, tokenizer

    def _get_gpu_memory(self) -> float:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.free', '--format=csv,noheader,nounits'],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        memory_free = result.stdout.decode().strip()
        try:
            return float(memory_free)
        except ValueError:
            return 0.0


In [49]:
llm= LLM()
model,tokenizer = llm()


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Available GPU Memory Before '__get_model': 11213.00 MB
Loading Quantized Model
Model found in 'loaded_model' directory. Loading model from there...
Available GPU Memory After '__get_model': 7867.00 MB


In [50]:
%%capture
!pip install git+https://github.com/uiuc-focal-lab/syncode.gitpred['intent'].lower().strip()

In [51]:
# from syncode import SyncodeLogitsProcessor

In [52]:
model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

In [53]:
chat = ChatCompletion(model=model, tokenizer=tokenizer)

messages = [
    {"role": "user", "content": "Hello! How are you?"},
    {"role": "assistant", "content": "I'm good, thank you! How can I assist you today?"}
]

response = chat.create(messages)
print("Response:", response)

Response: Is there something specific you'd like to talk about or ask about? I'm here to help with any questions or topics you'd like to discuss.


In [54]:
import json

def get_intent(model, tokenizer, system_message, query,device):
  """
  Extracts intent from the model's response.

  Args:
    model: The pre-trained language model.
    tokenizer: The tokenizer associated with the model.
    system_message: The system message for the chat.
    device: The device to run the model on (e.g., 'cuda' or 'cpu').

  Returns:
    A dictionary representing the extracted intent.
  """
  messages = [
      system_message,
      {"role": "user", "content": query},
  ]
  encoded_input = tokenizer.apply_chat_template(messages, tokenize = True,return_dict=True,return_tensors="pt").to(device)
  outputs = model.generate(
      **encoded_input,
      max_new_tokens=50,
      temperature=0.6,
      pad_token_id=tokenizer.eos_token_id,
      eos_token_id=tokenizer.eos_token_id,
      )
  output = tokenizer.decode(outputs[0])
  intent_pred = output.split('assistant<|end_header_id|>')[-1].strip().split('<|eot_id|>')[0].strip()

  try:
    intent_pred = json.loads(intent_pred)
  except Exception as e:
    print(f"Error parsing initial response: {e}\n{intent_pred}")
    messages = [
        {"role":'assistant','content':intent_pred},
        {"role": "user", "content": f"In your previous response {intent_pred} got this error {str(e)} \n make sure it is json response so that is should consume by automated system.Do not generate anything apart from json output as keys and value as same as {intent_pred}"},
    ]
    encoded_input = tokenizer.apply_chat_template(messages, tokenize = True,return_dict=True,return_tensors="pt").to(device)
    outputs = model.generate(
        **encoded_input,
        max_new_tokens=50,
        temperature=0.6,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(outputs[0])
    intent_pred = output.split('assistant<|end_header_id|>')[-1].strip().split('<|eot_id|>')[0].strip()
    print(f"Corrected response: {intent_pred}")

    try:
      intent_pred = json.loads(intent_pred)
    except:
      intent_pred = {'intent':'other'}

  return intent_pred

# intent = get_intent(model, tokenizer, system_message, model.device)
# print("Extracted Intent:", intent)

In [55]:
acc = 0
preds = []
for i,row in examples_df.iterrows():
  pred = get_intent(model, tokenizer, system_message, row['Examples'],model.device)
  if pred['intent'].lower().strip() == row['Intent'].lower().strip():
    acc += 1
  print(row['Examples'],pred['intent'].lower().strip(),row['Intent'])
  preds.append(pred['intent'].lower().strip())
print(f"Accuracy >- {acc/examples_df.shape[0]}")


I have a headache and a fever, what could it be? healthcare healthcare
My throat is really sore and I'm coughing, what should I do? healthcare healthcare
I woke up with a rash all over my body, is this contagious? healthcare healthcare
I've been feeling dizzy and nauseous lately, what could be the cause? healthcare healthcare
Can I book an appointment with Dr. Smith next Tuesday? healthcare healthcare
I need to see a cardiologist, how do I schedule a consultation? healthcare healthcare
What are the available slots for a pediatric dentist next week? healthcare healthcare
I need to reschedule my appointment with Dr. Jones, what should I do? healthcare healthcare
What are the side effects of ibuprofen? healthcare healthcare
Can I take this medication with antibiotics? healthcare healthcare
How long does it take for this medication to start working? healthcare healthcare
What is the dosage for this medicine for children? healthcare healthcare
What are the visiting hours for XYZ Hospital? h

In [56]:
acc

61

In [57]:
%%capture
!pip install gradio


In [58]:
system_prompt = """
You are an intelligent virtual assistant designed to assist users with queries in the following domains:

1. **Healthcare**: Answer questions about symptoms, treatments, medications, appointments ,etc and related to healthcare.
2. **Insurance**: Provide information on policies, claims, and coverage details and related to insurance industry.
3. **Finance**: Respond to queries about investments, loans, credit cards,etc and related to finance.
4. **Retail**: Assist with product availability, pricing, and order tracking etc related to retail industry.

---

**Out-of-Domain Handling**:
For queries outside these domains, respond with:
**"I can only assist with queries related to Healthcare, Insurance, Finance, or Retail."**

---

**Tone and Style**:
- Be professional, clear, and concise.
- Use simple language, especially for complex queries.
- Maintain a helpful and empathetic tone in sensitive topics like healthcare.

---

**Key Guidelines**:
- Provide accurate, verified information.
- For complex queries, break down responses into easy-to-understand steps.
- Prioritize user privacy—do not ask for sensitive personal details.
"""


In [59]:
prompt_template.chatbot_prompt = system_prompt
prompt_template.format()

{'role': 'system',
 'content': '\nYou are an intelligent virtual assistant designed to assist users with queries in the following domains:\n\n1. **Healthcare**: Answer questions about symptoms, treatments, medications, appointments ,etc and related to healthcare.\n2. **Insurance**: Provide information on policies, claims, and coverage details and related to insurance industry.\n3. **Finance**: Respond to queries about investments, loans, credit cards,etc and related to finance.\n4. **Retail**: Assist with product availability, pricing, and order tracking etc related to retail industry.\n\n---\n\n**Out-of-Domain Handling**:\nFor queries outside these domains, respond with:\n**"I can only assist with queries related to Healthcare, Insurance, Finance, or Retail."**\n\n---\n\n**Tone and Style**:\n- Be professional, clear, and concise.\n- Use simple language, especially for complex queries.\n- Maintain a helpful and empathetic tone in sensitive topics like healthcare.\n\n---\n\n**Key Guidel

In [60]:
import gradio as gr

chat_history = ChatHistory()
chat = ChatCompletion(model=model, tokenizer=tokenizer)
prompt_template.chatbot_prompt = system_prompt
bot_prpmt = prompt_template.format()
def chatbot_response(user_input: str):
    chat_history.add_message("user", user_input)

    messages = [bot_prpmt]+chat_history.chat_history

    bot_response = chat.create(messages)

    chat_history.add_message("assistant", bot_response)

    full_chat_history = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in chat_history.get_chat_history()])

    return full_chat_history, bot_response



In [61]:
# chatbot_response('hi')

In [62]:
iface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Textbox(label="Your Message", placeholder="Type a message...", lines=2),
        # gr.Textbox(label="Chat History", placeholder="Chat history will appear here...", lines=10, interactive=False)
    ],
    outputs=[
        gr.Textbox(label="Chat History", placeholder="Chat history will appear here...", lines=10, interactive=False),
        gr.Textbox(label="Bot Response", placeholder="Response from the bot...", lines=2, interactive=False)
    ],
    live=False,
    title="Chatbot Interface",
    description="Chat with the bot. Type your message and click Submit to get a response."
)
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff9f632e2b12153fbd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
